In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
import pandas as pd
import scipy.io as sio
import os
import shutil 
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [2]:
def mkdir(path):
    isExists = os.path.exists(path) 
    if not isExists:
        os.makedirs(path)
        return
    else:
        shutil.rmtree(path)
        os.makedirs(path)
        return

In [3]:
def combine_data_C(data, C):
    data_with_c = data.copy()
    data_with_c['C'] = C
    return data_with_c

In [4]:
def nor_data(data2, method):
    if method == 0:
        return (data2 - data2.min())/(data2.max() - data2.min()) ##rescaling
    if method == 1:
        return (data2 - data2.mean())/(data2.max() - data2.min()) ##mean normalization
    if method == 2:
        return (data2-data2.mean())/(data2.std()) ##standarization

In [12]:
def csv_group(inputfile, clusternumber, plotpath = "./outplot", method = 2, groupkey = "indtype", outcsvpath = "./outcsv/", SaveEachPlot = False):
    ###
    # 1. Read data as dataframe ###
    #
    data = pd.read_csv(inputfile)

    ###
    # 2.Grouped with keyvalue ###
    #
    data_to_count = data['indtype']
    intdic = {}
    for i in data_to_count: #count appe time
        cur_key = int(i/100)
        if cur_key in intdic:
            if i in intdic[cur_key]:
                intdic[cur_key][i] += 1
            else:
                intdic[cur_key][i] = 1
        else:
            intdic[cur_key] = {i:1}

    del_list = []
    count_dic = {}
    for i in intdic:
        summ = 0
        for j in intdic[i]:
            summ += intdic[i][j]
        if summ < clusternumber:
            del_list.append(i)
        else:
            count_dic[i] = summ

    ##drop small group
    for i in del_list:
        del intdic[i]

    order_dickey = list(intdic.keys())
    order_dickey.sort()
    
    
    ###
    # 5. Data Process
    pdfpath = inputfile + "_group_xx"+"_c" + str(clusternumber)  + ".pdf"
    data_feature = [] ##[group, giovsaled, perseng, voexpg, expinten, fa, ave_cwp]
    with PdfPages(pdfpath) as pdf:
        for i in order_dickey:
            avail_keygroup = list(intdic[i].keys())
            avail_keygroup.sort()
            data_to_processed = data[data['indtype'] == avail_keygroup[0]]
            for j in avail_keygroup[1:]:
                df2 = data[data['indtype'] == j]
                data_to_processed = data_to_processed.append(df2)

            data_to_processed['ave_cwp'] = data_to_processed['cwp'] / data_to_processed['perseng']

            ##5.2
            data2 = pd.DataFrame(data_to_processed, columns = ['rndinten', 'expinten'])
            ss = StandardScaler()
            data2 = ss.fit_transform(data2)

            ##5.3
            sk_kmeans = KMeans(n_clusters=clusternumber)
            sk_kmeans.fit(data2)

            ##5.4produce cluster label
            sk_C = sk_kmeans.predict(data2)
            data_with_c = combine_data_C(data_to_processed, sk_C)

            ##5.5 Save Plot
            sns.lmplot('rndinten', 'expinten', hue='C', data=data_with_c, fit_reg=False)
            plt.title("indtype = "+str(i)+"xx")
            plt.tight_layout()
            pdf.savefig()
            if (SaveEachPlot):
                plt.savefig(plotpath + str(intkey)+".png", dpi = 200)
            plt.close()
    
            df1 = data_with_c[data_with_c['C'] == 0 ]
            df2 = data_with_c[data_with_c['C'] == 1 ]

            data_feature.append([str(i)+'xx','c0',df1['giovsaled'].mean(), df1['perseng'].mean(), df1['voexpg'].mean(), 
                                df1['expinten'].mean(), df1['fa'].mean(), df1['ave_cwp'].mean()])
            
            data_feature.append([str(i)+'xx','c1',df2['giovsaled'].mean(), df2['perseng'].mean(), df2['voexpg'].mean(), 
                                df2['expinten'].mean(), df2['fa'].mean(), df2['ave_cwp'].mean()])
            
            

    d_feature = pd.DataFrame(data_feature, columns = ['group', 'class','ave_giovsaled', 'ave_perseng', 'ave_voexpg', 'ave_expinten', 'ave_fa', 'ave_cwp'])
    d_feature.to_csv('data_feature.csv', index = 0)
    return 

In [13]:
infile = "2001_big.csv"
num = 2
csv_group(infile, num)

In [ ]:
! git add .
! git commit -m "2.3update1"
! git push